In [10]:
import concurrent.futures
import random
import time
import typing

import polars as pl
import plotnine as p9

from tools.constants import (
    AMINO_ACID_TO_CODONS_MAP,
    CODON_TO_AMINO_ACID_MAP,
    SEQUENCE_SARSCOV2_SPIKE,
    SEQUENCE_EGFP,
    SEQUENCE_LUCIFERASE,
)
from tools.sequence.sequence import Sequence
from tools.types import Organism
from tools.data import load_codon_usage_table

In [11]:
PROTEINS = {
    "EGFP": SEQUENCE_EGFP,
    "LUCIFERASE": SEQUENCE_LUCIFERASE,
    "SARSCOV2_SPIKE": SEQUENCE_SARSCOV2_SPIKE,
    "CAS9_PE": "MKRTADGSEFESPKKKRKVDKKYSIGLDIGTNSVGWAVITDEYKVPSKKFKVLGNTDRHSIKKNLIGALLFDSGETAEATRLKRTARRRYTRRKNRICYLQEIFSNEMAKVDDSFFHRLEESFLVEEDKKHERHPIFGNIVDEVAYHEKYPTIYHLRKKLVDSTDKADLRLIYLALAHMIKFRGHFLIEGDLNPDNSDVDKLFIQLVQTYNQLFEENPINASGVDAKAILSARLSKSRRLENLIAQLPGEKKNGLFGNLIALSLGLTPNFKSNFDLAEDAKLQLSKDTYDDDLDNLLAQIGDQYADLFLAAKNLSDAILLSDILRVNTEITKAPLSASMIKRYDEHHQDLTLLKALVRQQLPEKYKEIFFDQSKNGYAGYIDGGASQEEFYKFIKPILEKMDGTEELLVKLNREDLLRKQRTFDNGSIPHQIHLGELHAILRRQEDFYPFLKDNREKIEKILTFRIPYYVGPLARGNSRFAWMTRKSEETITPWNFEEVVDKGASAQSFIERMTNFDKNLPNEKVLPKHSLLYEYFTVYNELTKVKYVTEGMRKPAFLSGEQKKAIVDLLFKTNRKVTVKQLKEDYFKKIECFDSVEISGVEDRFNASLGTYHDLLKIIKDKDFLDNEENEDILEDIVLTLTLFEDREMIEERLKTYAHLFDDKVMKQLKRRRYTGWGRLSRKLINGIRDKQSGKTILDFLKSDGFANRNFMQLIHDDSLTFKEDIQKAQVSGQGDSLHEHIANLAGSPAIKKGILQTVKVVDELVKVMGRHKPENIVIEMARENQTTQKGQKNSRERMKRIEEGIKELGSQILKEHPVENTQLQNEKLYLYYLQNGRDMYVDQELDINRLSDYDVDAIVPQSFLKDDSIDNKVLTRSDKNRGKSDNVPSEEVVKKMKNYWRQLLNAKLITQRKFDNLTKAERGGLSELDKAGFIKRQLVETRQITKHVAQILDSRMNTKYDENDKLIREVKVITLKSKLVSDFRKDFQFYKVREINNYHHAHDAYLNAVVGTALIKKYPKLESEFVYGDYKVYDVRKMIAKSEQEIGKATAKYFFYSNIMNFFKTEITLANGEIRKRPLIETNGETGEIVWDKGRDFATVRKVLSMPQVNIVKKTEVQTGGFSKESILPKRNSDKLIARKKDWDPKKYGGFDSPTVAYSVLVVAKVEKGKSKKLKSVKELLGITIMERSSFEKNPIDFLEAKGYKEVKKDLIIKLPKYSLFELENGRKRMLASAGELQKGNELALPSKYVNFLYLASHYEKLKGSPEDNEQKQLFVEQHKHYLDEIIEQISEFSKRVILADANLDKVLSAYNKHRDKPIREQAENIIHLFTLTNLGAPAAFKYFDTTIDRKRYTSTKEVLDATLIHQSITGLYETRIDLSQLGGDSGGSSGGSSGSETPGTSESATPESSGGSSGGSSTLNIEDEYRLHETSKEPDVSLGSTWLSDFPQAWAETGGMGLAVRQAPLIIPLKATSTPVSIKQYPMSQEARLGIKPHIQRLLDQGILVPCQSPWNTPLLPVKKPGTNDYRPVQDLREVNKRVEDIHPTVPNPYNLLSGLPPSHQWYTVLDLKDAFFCLRLHPTSQPLFAFEWRDPEMGISGQLTWTRLPQGFKNSPTLFNEALHRDLADFRIQHPDLILLQYVDDLLLAATSELDCQQGTRALLQTLGNLGYRASAKKAQICQKQVKYLGYLLKEGQRWLTEARKETVMGQPTPKTPRQLREFLGKAGFCRLFIPGFAEMAAPLYPLTKPGTLFNWGPDQQKAYQEIKQALLTAPALGLPDLTKPFELFVDEKQGYAKGVLTQKLGPWRRPVAYLSKKLDPVAAGWPPCLRMVAAIAVLTKDAGKLTMGQPLVILAPHAVEALVKQPPDRWLSNARMTHYQALLLDTDRVQFGPVVALNPATLLPLPEEGLQHNCLDILAEAHGTRPDLTDQPLPDADHTWYTDGSSLLQEGQRKAGAAVTTETEVIWAKALPAGTSAQRAELIALTQALKMAEGKKLNVYTDSRYAFATAHIHGEIYRRRGWLTSEGKEIKNKDEILALLKALFLPKRLSIIHCPGHQKGHSAEARGNRMADQAARKAAITETPDTSTLLIENSSPSGGSKRTADGSEFEGGGGSGGGGSGGGGSMVSKGEAVIKEFMRFKVHMEGSMNGHEFEIEGEGEGRPYEGTQTAKLKVTKGGPLPFSWDILSPQFMYGSRAFTKHPADIPDYYKQSFPEGFKWERVMNFEDGGAVTVTQDTSLEDGTLIYKVKLRGTNFPPDGPVMQKKTMGWEASTERLYPEDGVLKGDIKMALRLKDGGRYLADFKTTYKAKKPVQMPGAYNVDRKLDITSHNEDYTVVEQYERSEGRHSTGGMDELYKPKKKRKVPKKKRKV",
}
NUM_SEQUENCES_PER_CAI = 3
CAI_RANGE = [
    0.90,
    0.92,
    0.94,
    0.96,
    0.98,
    1.0,
]
CAI_LIMITS = [
    0.02,
    0.04,
    0.06,
    0.08,
    0.1,
    0.12,
]

for name, protein in PROTEINS.items():
    print(f"{name}: {len(protein)} nt")


def _max_cai_sequence(s: Sequence, organism: Organism = "homo-sapiens") -> Sequence:
    codon_usage_table = load_codon_usage_table(organism)
    new_codons = []
    for codon in s.codons:
        amino_acid = CODON_TO_AMINO_ACID_MAP[codon]
        codon_usage = codon_usage_table.most_frequent(amino_acid)
        new_codons.append(codon_usage.codon)
    return Sequence.from_string("".join(new_codons))


def _min_cai_sequence(s: Sequence, organism: Organism = "homo-sapiens") -> Sequence:
    codon_usage_table = load_codon_usage_table(organism)
    new_codons = []
    for codon in s.codons:
        amino_acid = CODON_TO_AMINO_ACID_MAP[codon]
        codon_usage = codon_usage_table.least_frequent(amino_acid)
        new_codons.append(codon_usage.codon)
    return Sequence.from_string("".join(new_codons))


def _estimate_mfe(s: Sequence, window: int = 40, step: int = 1) -> float:
    mfes = [
        s[i : i + window].minimum_free_energy.energy
        for i in range(0, len(s) - window, step)
    ]
    return sum(mfes) / len(mfes)


def _mutate(sequence: Sequence, mutations_per_iteration: int) -> Sequence:
    next_sequence = Sequence.from_string(str(sequence))
    for _ in range(mutations_per_iteration):
        location = random.randrange(0, len(next_sequence), 3)
        codon = str(next_sequence[location : location + 3])
        amino_acid = CODON_TO_AMINO_ACID_MAP[codon]
        next_codon = random.choice(list(AMINO_ACID_TO_CODONS_MAP[amino_acid]))
        next_sequence = (
            next_sequence[:location]
            + Sequence.from_string(next_codon)
            + next_sequence[location + 3 :]
        )
    return next_sequence


def _optimize_cai(
    name: str,
    sequence: Sequence,
    target_cai: float,
    organism: Organism = "homo-sapiens",
    iterations: int = 20_000,
    mutations_per_iteration: int = 2,
) -> Sequence:
    best_sequence = sequence
    best_cai = best_sequence.codon_adaptation_index(organism)
    for _ in range(iterations):
        next_sequence = _mutate(best_sequence, mutations_per_iteration)
        next_cai = next_sequence.codon_adaptation_index(organism)
        if abs(next_cai - target_cai) < abs(best_cai - target_cai):
            best_sequence = next_sequence
            best_cai = next_cai
        if abs(best_cai - target_cai) / target_cai < 0.002:
            # Quit early if very close to target CAI (within 0.2%)
            break
    return best_sequence


def _optimize_mfe(
    name: str,
    sequence: Sequence,
    cai_limit: float,
    direction: typing.Literal["MAX", "MIN"],
    iterations: int = 1000,
    mutations_per_iteration: int = 2,
) -> Sequence:
    best_sequence = sequence
    best_cai = best_sequence.codon_adaptation_index()
    best_mfe = _estimate_mfe(best_sequence, step=4)
    for _ in range(iterations):
        next_sequence = _mutate(best_sequence, mutations_per_iteration)
        next_cai = next_sequence.codon_adaptation_index()
        if abs(next_cai - best_cai) / best_cai > cai_limit:
            continue
        next_mfe = _estimate_mfe(next_sequence, step=4)
        if direction == "MIN" and next_mfe < best_mfe:
            best_sequence = next_sequence
            best_mfe = next_mfe
        if direction == "MAX" and next_mfe > best_mfe:
            best_sequence = next_sequence
            best_mfe = next_mfe
    return best_sequence


def _optimize(name: str, sequence: Sequence, cai_limit: float, optimize_mfe: str):
    start = time.perf_counter()
    if optimize_mfe:
        optimized = _optimize_mfe(name, sequence, cai_limit, optimize_mfe)
    else:
        optimized = _optimize_cai(name, sequence, 1.0 - cai_limit)
    return {
        "name": name,
        "cai_limit": cai_limit,
        "output_sequence": str(optimized),
        "optimize_mfe": optimize_mfe,
        "time": time.perf_counter() - start,
    }


max_cai_sequences = {
    name: _max_cai_sequence(Sequence.from_string(s)) for name, s in PROTEINS.items()
}

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [
        executor.submit(_optimize, name, s, cai_limit, optimize_mfe)
        for name, s in max_cai_sequences.items()
        # for target_cai in CAI_RANGE
        for cai_limit in CAI_LIMITS
        for _ in range(NUM_SEQUENCES_PER_CAI)
        for optimize_mfe in ["MAX", "MIN", None]
    ]
    print(len(futures))
    done, not_done = concurrent.futures.wait(futures)
    assert not not_done
    results = [d.result() for d in done]

df = pl.DataFrame(results).with_columns(
    pl.col("output_sequence")
    .map_elements(
        lambda s: Sequence.from_string(s).codon_adaptation_index(),
        return_dtype=pl.Float64,
    )
    .alias("output_cai")
)
df

EGFP: 239 nt
LUCIFERASE: 550 nt
SARSCOV2_SPIKE: 1273 nt
CAS9_PE: 2371 nt
216


name,cai_limit,output_sequence,optimize_mfe,time,output_cai
str,f64,str,str,f64,f64
"""EGFP""",0.06,"""ATGGTGAGCAAGGGCGAGGAGTTATTCACG…",null,0.066582,0.939103
"""SARSCOV2_SPIKE""",0.06,"""ATGTTCGTGTTCCTGGTGCTGCTGCCCCTG…",null,1.104039,0.941502
"""SARSCOV2_SPIKE""",0.04,"""ATGTTTGTGTTCCTGGTGCTGCTGCCTCTG…","""MIN""",17.647091,0.960153
"""LUCIFERASE""",0.02,"""ATGGAGGACGCCAAGAACATCAAGAAGGGC…",null,0.064608,0.981588
"""SARSCOV2_SPIKE""",0.04,"""ATGTTCGTGTTCCTGGTGCTGCTGCCCCTG…","""MAX""",14.588045,0.960056
…,…,…,…,…,…
"""EGFP""",0.06,"""ATGGTGAGCAAGGGCGAGGAGCTGTTCACC…","""MIN""",2.755632,0.940352
"""LUCIFERASE""",0.04,"""ATGGAGGACGCCAAGAACATCAAGAAGGGC…",null,0.058994,0.96181
"""SARSCOV2_SPIKE""",0.08,"""ATGTTCGTGTTCCTGGTGCTGCTGCCCCTG…","""MAX""",15.439921,0.920026


In [12]:
df.write_csv("genetic.csv")